In [1]:
import re
import ast 
import json
import pandas as pd
import import_ipynb
import sql_operations
from jsonpath_ng.ext import parse

importing Jupyter notebook from sql_operations.ipynb


In [2]:
file_name = 'openapi.json'

In [3]:
def json_to_sql(file_name):
    #loading json data
    data = json.load(open(file_name))
    
    # Define your JSONPath expressions
    jsonpath_expr = parse("$[*]")
    match = [match.value for match in jsonpath_expr.find(data)]
    
    #create cursor
    conn, cursor = sql_operations.create_cursor()
    
    #delete tables if existing
    sql_operations.delete_tables(cursor)
    
    #create sql tables
    sql_operations.create_tables(cursor)
    
    for item in match:
        # Extract data using JSONPath expressions
        name_match = item['name']
        type_match = item['type']
        genre_match = item['genre']
        date_match = item['released_at']
        plat_match = item['streaming_on']
        ctry_match = item['country']
        rating_match = item['imdb_rating']
        cast = ast.literal_eval(item['cast_and_crew'])
        director = None
        actors = []
        # Iterate through the cast_and_crew list
        for tem in cast:
            name = tem["name"]
            description = tem["description"]
            if description == "director":
                director = name
            elif description == "actor":
                actors.append(name)
        # Insert data into the table
        insert_query1 = """
        INSERT INTO movies (name, type, director, release_date, genre)
        VALUES (?, ?, ?, ?, ?);
        """
        insert_query2 = """
        INSERT INTO stream_info (name, platform, country)
        VALUES (?, ?, ?);
        """
        insert_query3 = """
        INSERT INTO rating (name, rating)
        VALUES (?, ?);
        """
        insert_query4 = """
        INSERT INTO actors (name, actor)
        VALUES (?, ?);
        """
        cursor.execute(insert_query1, (name_match, type_match, director, date_match, genre_match))
        cursor.execute(insert_query2, (name_match, plat_match, ctry_match))
        cursor.execute(insert_query3, (name_match, rating_match))
        for actor in actors:
            cursor.execute(insert_query4, (name_match, actor))


    # Commit the changes and close the database connection
    conn.commit()
    return cursor

In [6]:
cursor = json_to_sql(file_name)

In [7]:
cursor.execute("SELECT * FROM actors;")

In [8]:
print(cursor.fetchall())

[(1, 'Mike Epps: Under Rated & Never Faded', 'Mike Epps'), (2, 'Mike Epps: Under Rated & Never Faded', 'Doug E. Fresh'), (3, 'Star Trek III: The Search for Spock', 'William Shatner'), (4, 'Star Trek III: The Search for Spock', 'DeForest Kelley'), (5, 'Star Trek III: The Search for Spock', 'Robin Curtis'), (6, 'Star Trek III: The Search for Spock', 'Merritt Butrick'), (7, 'Star Trek III: The Search for Spock', 'Christopher Lloyd'), (8, 'Star Trek III: The Search for Spock', 'James Doohan'), (9, 'Star Trek III: The Search for Spock', 'George Takei'), (10, 'Star Trek III: The Search for Spock', 'Walter Koenig'), (11, 'Star Trek III: The Search for Spock', 'Nichelle Nichols'), (12, 'Star Trek III: The Search for Spock', 'Mark Lenard'), (13, 'Star Trek III: The Search for Spock', 'James B. Sikking'), (14, 'Star Trek III: The Search for Spock', 'Phillip R. Allen'), (15, 'Star Trek III: The Search for Spock', 'Robert Hooks'), (16, 'Star Trek III: The Search for Spock', 'John Larroquette'), (1